# Лабораторная работа 1.
### Вариант 1

Работа выполнена студентом группы М1О-415Бки-19 Кравченко Д.В.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os

dataset = pd.read_csv("datasets/breast_cancer.csv")
dataset = dataset.drop(columns = 'Unnamed: 32')

In [ ]:
dataset.describe()

## Распределение величин
Рассмотрим распределение параметров в зависимости от диагноза, где красным показан диагноз **М** (злокачественная опухоль), а синим - дигноз **B** (доброкачественная опухоль):

In [ ]:
%matplotlib inline
width = 5
height = 6

fig, axs = plt.subplots(width, height, figsize=(25,20))

for i in range(2,31):
    x = (i - 2) % width
    y = (i - 2) // width
    series_b = dataset[dataset["diagnosis"] == "M"][dataset.columns[i]]
    series_m = dataset[dataset["diagnosis"] == "B"][dataset.columns[i]]
    axs[x, y].hist(series_b, color = 'blue', alpha = 0.8, density = True, stacked = True)
    axs[x, y].hist(series_m, color = 'red', alpha = 0.8, density = True, stacked = True)
    axs[x, y].set_title(dataset.columns[i])
fig.tight_layout()
plt.show()


Как видно по графикам, наиболее сильно с диагнозом коррелируют следующие параметры:
 - ```radius_mean```
 - ```radius_se```
 - ```radius_worst```
 - ```area_mean```
 - ```area_se```
 - ```area_worst```
 - ```compactness_mean```
 - ```concavity_mean```
 - ```concave_points_mean```
 - ```perimeter_worst```

Проверим подробнее корреляцию между диагнозом и параметрами.

## Корреляция
Рассмотрим корреляцию между диагнозом и остальными параметрами. Для рассчета корреляции строковый параметр ```diagnosis``` необходимо преобразовать в тип категории, где ```0``` - означает диагноз **В**, а ```1``` - диагноз **М**.

In [ ]:
dataset["diagnosis"] = dataset["diagnosis"].astype('category').cat.codes
corr_matrix = dataset.loc[:,dataset.columns!='id'].corr(numeric_only=True)
corr_matrix["diagnosis"].sort_values(ascending=False)

Рассмотрим зависимости между наиболее коррелирующими с диагнозом параметрами:

In [ ]:
from pandas.plotting import scatter_matrix

important_features = [column for column in dataset.columns if column not in ['id', 'diagnosis'] and abs(corr_matrix['diagnosis'][column]) > 0.5]
scatter_matrix(dataset[important_features],figsize=(20,20));

Можно заметить достаточно четкие зависимости между параметрами:
 - линейную зависимость между ```radius_mean``` и ```perimeter_mean```
 - линейную зависимость между ```radius_worst``` и ```perimeter_worst```
 - квадратичную зависимость между ```area_mean``` и ```radius_mean```/```perimeter_mean```
 - квадратичную зависимосит между ```area_worst``` и ```radius_worst```/```perimeter_worst```
Также можно заметить множество линейных и квадратичных зависимостей между другими параметрами, однако эти зависимости выражены гораздо менее явно.

Проверим коэффициенты корреляции:

In [ ]:
(corr_matrix["radius_mean"]["perimeter_mean"], corr_matrix["radius_worst"]["perimeter_worst"])

На основании этого можно сделать вывод, что некоторые параметры являются избыточными.

Рассмотрим зависимость между диагнозом и наиболее коррелирующими параметрами:

In [ ]:
ax = dataset[dataset["diagnosis"] == 1].plot.scatter(x="concave points_worst", y="perimeter_worst", color="Red", label="M", alpha = 0.6)
dataset[dataset["diagnosis"] == 0].plot.scatter(x="concave points_worst", y="perimeter_worst", color="Blue", label="B", alpha = 0.6, ax=ax);

Выберем из сипска параметров наиболее коррелирующие с диагнозом:

In [ ]:
unimportant_features = ['id', 'diagnosis', 'radius_worst', 'radius_mean']
important_features = [column for column in dataset.columns if column not in unimportant_features and abs(corr_matrix['diagnosis'][column]) > 0.5]
important_features

## Создание обучающей и тестовой выборок
Разделим датасет на стратифицированные выборки. После избавимся от пропусков в тестовой выборке (если есть)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
seed = 1989
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

for train_index, test_index in sss.split(dataset, dataset["diagnosis"]):
    train_set = dataset.loc[train_index]
    test_set = dataset.loc[test_index]
print(f"Train set size:\t{len(train_set)}\nTest set size:\t{len(test_set)}")

test_labels = test_set['diagnosis'].copy()
test_data = test_set.drop(['id', 'diagnosis'], axis=1)

init_size = len(train_set)
labels = train_set['diagnosis'].copy()
data = train_set.drop(['id', 'diagnosis'], axis=1)
print(f'There are {init_size - len(data)} rows with empty fields among {init_size} rows.')

Поставленная задача относится к задачам классификации - на основании вводимых параметров решать, принадлежит ли пациент к классу с диагнозом **В** или **М**.
Попробуем решить задачу несколькими способами.

## Дерево решений
В связи с элементом случайности, проведем несколько тестов и оценим статистику в целом.

In [ ]:
from sklearn import tree
from scipy import stats
scores = np.empty([100], dtype=float)
for i in range(0,100):
    tree_cls = tree.DecisionTreeClassifier()
    tree_cls = tree_cls.fit(data, labels)

    scores[i] = tree_cls.score(test_data, test_labels)
stats.describe(scores)

## Линейный классификатор
Путем приведения классов **В** и **М** к значениям {-1, 1} можно привести задачу классификации к задаче регрессии. В ```sklearn``` представлено несколько методов решения такой задачи - к примеру, аргумент ```lsqr``` решает задачу с использованием метода наименьших квадратов, а ```sag``` и ```saga``` используют стохастический градиентный спуск.

In [ ]:
from sklearn import linear_model
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
scores = []
for solver in solvers:
    reg = linear_model.RidgeClassifier(solver=solver)
    reg.fit(data, labels)
    scores.append(reg.score(test_data, test_labels))
for solver, score in zip(solvers, scores):
    print(f'{solver:<10}: {score}')

Теперь оценим работу вышеперечисленных методов при откидывании избыточных параметров:

In [ ]:
reduced_train_data = train_set.copy().drop(unimportant_features, axis = 1)
reduced_train_labels = train_set['diagnosis']
reduced_test_data = test_set.copy().drop(unimportant_features, axis = 1)
reduced_test_labels = test_set['diagnosis']

## Дерево решений

In [ ]:
from sklearn import tree
from scipy import stats
scores = np.empty([100], dtype=float)
for i in range(0,100):
    tree_cls = tree.DecisionTreeClassifier()
    tree_cls = tree_cls.fit(reduced_train_data, reduced_train_labels)

    scores[i] = tree_cls.score(reduced_test_data, reduced_test_labels)
stats.describe(scores)

## Линейный классификатор

In [ ]:
from sklearn import linear_model
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
scores = []
for solver in solvers:
    reg = linear_model.RidgeClassifier(solver=solver)
    reg.fit(reduced_train_data, reduced_train_labels)
    scores.append(reg.score(reduced_test_data, reduced_test_labels))
for solver, score in zip(solvers, scores):
    print(f'{solver:<10}: {score}')

Как можно увидеть из результатов, максимальной точностью является значение в ```95.61403%```. Этот результат является достаточно низким, что скорее всего обусловлено достаточно малым размером датасета.
Помимо этого в выборке есть некоторые параметры, которые можно посчитать избыточными, однако их удаление ведет к ощутимому падению точности.